# Kepler Framework, Emulate v17

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'


In [4]:
tpath_data17     = '../v12/output/emulation/data17_table_v12.h5'
tpath_data17_trk = '../v3_el/output/emulation/data17_table_v3_el.h5'


tpath_data18     = '../v12/output/emulation/data18_table_v12.h5'
tpath_data18_trk = '../v3_el/output/emulation/data18_table_v3_el.h5'




## Setup Chains:

In [5]:
# create my chain
chains = [
    Chain( "HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI"    , L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v12_vloose',l2_column = 'ringer_v3_el_loose'),
    Chain( "HLT_e28_lhtight_nod0_ringer_v17_ivarloose"     , L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v12_tight' ,l2_column = 'ringer_v3_el_loose'),
    Chain( "HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI"    , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v12_medium',l2_column = 'ringer_v3_el_loose'),
    Chain( "HLT_e140_lhloose_nod0_ringer_v17"              , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v12_loose' ,l2_column = 'ringer_v3_el_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v12_vloose')),
             ('L2', 'ringer_v3_el_loose'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v12_tight')),
             ('L2', 'ringer_v3_el_loose'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v12_medium')),
             ('L2', 'ringer_v3_el_loose'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v12_loose')),
             ('L2', 'ringer_v3_el_loose'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [7]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17), load_hdf(tpath_data17_trk)), axis=1)

In [8]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:48<00:00, 27.01s/it]


In [9]:
data17_df.shape

(43311283, 114)

### Validate:

In [10]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v17.root')

2022-03-26 01:25:04,397 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v17.root


In [12]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v17_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v17'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:54<00:00, 22.98s/it]


HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5258  | 29972729/32047563 |
|   L2   | 92.9344  | 29783205/32047563 |
| EFCalo | 91.6687  | 29377572/32047563 |
|  HLT   | 89.8077  | 28781181/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.43s/it]


HLT_e28_lhtight_nod0_ringer_v17_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2296  | 21451925/22765602 |
|   L2   | 94.0847  | 21418937/22765602 |
| EFCalo | 93.1712  | 21210991/22765602 |
|  HLT   | 87.0169  | 19809928/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.4190  | 1332484/1367786 |
|   L2   | 97.2633  | 1330354/1367786 |
| EFCalo | 95.1937  | 1302046/1367786 |
|  HLT   | 91.9230  | 1257310/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.25it/s]


HLT_e140_lhloose_nod0_ringer_v17
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 98.0724  | 59374/60541  |
|   L2   | 97.9072  | 59274/60541  |
| EFCalo | 96.6023  | 58484/60541  |
|  HLT   | 94.6516  | 57303/60541  |
+--------+----------+--------------+


In [13]:
eff_data17_Zee.save()

In [14]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v17.root')

2022-03-26 01:32:08,642 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v17.root


In [15]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v17_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v17'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.34s/it]


HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  2.2751  | 235470/10349992  |
|   L2   |  0.8707  |  90114/10349992  |
| EFCalo |  0.7761  |  80323/10349992  |
|  HLT   |  0.1145  |  11853/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.92s/it]


HLT_e28_lhtight_nod0_ringer_v17_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  1.3860  |  128264/9254300 |
|   L2   |  0.6451  |  59704/9254300  |
| EFCalo |  0.5473  |  50651/9254300  |
|  HLT   |  0.0019  |   180/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.32s/it]


HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  1.5870  |  124617/7852181 |
|   L2   |  0.6682  |  52472/7852181  |
| EFCalo |  0.5540  |  43502/7852181  |
|  HLT   |  0.0027  |   209/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.46s/it]

HLT_e140_lhloose_nod0_ringer_v17
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  2.4542  |  138268/5634041 |
|   L2   |  0.8651  |  48740/5634041  |
| EFCalo |  0.7390  |  41634/5634041  |
|  HLT   |  0.0331  |   1867/5634041  |
+--------+----------+-----------------+


In [16]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [8]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18), load_hdf(tpath_data18_trk)), axis=1)

In [9]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:41<00:00, 25.27s/it]


In [10]:
data18_df.shape

(41873973, 114)

In [11]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v17.root')

2022-03-26 01:48:40,012 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v17.root


In [12]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v17_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v17'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.52s/it]


HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5744  | 30524429/32620495 |
|   L2   | 93.0058  | 30338962/32620495 |
| EFCalo | 91.7160  | 29918220/32620495 |
|  HLT   | 90.3152  | 29461271/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.52s/it]


HLT_e28_lhtight_nod0_ringer_v17_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2032  | 21962376/23313826 |
|   L2   | 94.0796  | 21933554/23313826 |
| EFCalo | 93.2890  | 21749246/23313826 |
|  HLT   | 87.5932  | 20421332/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.20s/it]


HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.4305  | 1346173/1381675 |
|   L2   | 97.2743  | 1344014/1381675 |
| EFCalo | 95.2070  | 1315451/1381675 |
|  HLT   | 92.2251  | 1274251/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.14it/s]


HLT_e140_lhloose_nod0_ringer_v17
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.1288  | 59259/60389  |
|   L2   | 97.9384  | 59144/60389  |
| EFCalo | 96.7825  | 58446/60389  |
|  HLT   | 94.9494  | 57339/60389  |
+--------+----------+--------------+


In [13]:
eff_data18_Zee.save()

In [14]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v17.root')

2022-03-26 01:52:57,947 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v17.root


In [15]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v17_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v17'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.56s/it]


HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  2.7154  |  229916/8467098 |
|   L2   |  1.2809  |  108456/8467098 |
| EFCalo |  1.1911  |  100852/8467098 |
|  HLT   |  0.5459  |  46224/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.21s/it]


HLT_e28_lhtight_nod0_ringer_v17_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  1.8195  |  138807/7628804 |
|   L2   |  1.0382  |  79204/7628804  |
| EFCalo |  0.9464  |  72196/7628804  |
|  HLT   |  0.0501  |   3822/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.75s/it]


HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  2.0125  |  132059/6561985 |
|   L2   |  1.0516  |  69007/6561985  |
| EFCalo |  0.9458  |  62065/6561985  |
|  HLT   |  0.0625  |   4103/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.07s/it]

HLT_e140_lhloose_nod0_ringer_v17
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  2.9726  |  141989/4776518 |
|   L2   |  1.3439  |  64192/4776518  |
| EFCalo |  1.2239  |  58461/4776518  |
|  HLT   |  0.3286  |  15698/4776518  |
+--------+----------+-----------------+


In [16]:
eff_data18_JF17.save()